In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context

context = Context()

In [ ]:
from contracts.staking_contract import StakingContract
staking_contract: StakingContract
staking_contract = context.get_contracts(config.STAKINGS_BOOSTED)[0]

In [ ]:
from contracts.metastaking_contract import MetaStakingContract
metastaking_contract: MetaStakingContract
metastaking_contract = context.get_contracts(config.METASTAKINGS_BOOSTED)[0]

In [ ]:
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

In [ ]:
energy_contract.add_sc_to_whitelist(context.deployer_account, context.network_provider.proxy, 
                                    context.get_farm_contract_by_address("erd1qqqqqqqqqqqqqpgqaathw30vz468zxukyjzxc4aqnzrd2n5u295q4l9ycs").address)

In [ ]:
energy_contract.resume(context.deployer_account, context.network_provider.proxy)

In [ ]:
energy_contract.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [])

In [ ]:
txhash = staking_contract.register_farm_token(context.deployer_account, context.network_provider.proxy, ["STKTST", "STKTST"])
status = context.network_provider.check_simple_tx_status(txhash, "set boosted yields factors in farm")

In [ ]:
status = context.network_provider.check_simple_tx_status("5f5e3b111bc5ca2f59e4f8bb8faf3aa7e7827bc0247808b53eb7bd58e9db19e4", "set boosted yields factors in farm")

In [ ]:
from utils.contract_data_fetchers import MetaStakingContractDataFetcher
from utils.utils_chain import WrapperAddress as Address, hex_to_string
farm_token_hex = MetaStakingContractDataFetcher(Address(metastaking_contract.address), context.network_provider.proxy.url).get_data("getDualYieldTokenId")
print(hex_to_string(farm_token_hex))

In [ ]:
from contracts.pair_contract import PairContract
pair_contract: PairContract
pair_contracts = context.get_contracts(config.PAIRS_V2)
for pair_contract in pair_contracts:
    pair_contract.resume(context.deployer_account, context.network_provider.proxy)

In [ ]:
from contracts.router_contract import RouterContract
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]
router_contract.set_fee_on(context.deployer_account, context.network_provider.proxy, 
                           [
                                "erd1qqqqqqqqqqqqqpgqkdxw8ewquxtt64t8rmdgqz3ja0zjycfm295qpdm8at",
                                "erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu",
                                "MEX-4638e4"
                           ])

FEES setup for pairs

In [ ]:
import time
from contracts.pair_contract import PairContract
from contracts.router_contract import RouterContract

router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]

pair_contract: PairContract
mex_contract = context.get_pair_contract_by_address("erd1qqqqqqqqqqqqqpgqsa0hwt0m05308mxlagc6wm3qs6duuh5f295qs5du2a") # egldmex contract

for sending_contract in context.get_contracts(config.PAIRS_V2):
    if sending_contract.address in ["erd1qqqqqqqqqqqqqpgqsa0hwt0m05308mxlagc6wm3qs6duuh5f295qs5du2a",
                                    "erd1qqqqqqqqqqqqqpgqkdxw8ewquxtt64t8rmdgqz3ja0zjycfm295qpdm8at"]:
        continue
    # set where to swap and what to do with the fees
    sending_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                        [
                                            mex_contract.address,
                                            mex_contract.firstToken,
                                            mex_contract.secondToken
                                        ])
    time.sleep(2)
    sending_contract.set_fee_on_via_router(context.deployer_account, context.network_provider.proxy, router_contract, 
                                    [
                                        "erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu",
                                            "MEX-4638e4"
                                    ])
    time.sleep(2)
    # whitelist swap no fees
    mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, sending_contract.address)
    time.sleep(2)

In [ ]:
from contracts.router_contract import RouterContract
router_contract: RouterContract
router_contract = context.get_contracts(config.ROUTER_V2)[0]
mex_contract.set_fee_on_via_router(context.deployer_account, context.network_provider.proxy, router_contract, 
                                   [
                                       "erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu",
                                        "MEX-4638e4"
                                   ])